# Testing eFact-OSE Integration

Este notebook prueba la integración con eFact-OSE:
- OAuth2 Authentication
- JSON-UBL 2.1 Generation
- Number to Words Conversion
- Document Validations

## Setup - Import modules

In [1]:
# Configurar sys.path para poder importar desde app
import sys
from pathlib import Path

# Agregar el directorio backend al path
backend_path = Path.cwd()
if backend_path.name == "backend":
    sys.path.insert(0, str(backend_path))
else:
    # Si estamos en el root del monorepo, navegar a backend
    backend_path = backend_path / "apps" / "backend"
    sys.path.insert(0, str(backend_path))

print(f"✓ Backend path added: {backend_path}")

# Importar los módulos
from datetime import datetime
from app.integrations.efact_client import (
    efact_client,
    generate_json_ubl,
    numero_a_letras,
    validar_ruc,
    validar_dni,
    EFactError,
    EFactAuthError,
)
import json

print("✓ Modules imported successfully")

✓ Backend path added: c:\Users\Renzo\Desktop\Proyectos\Ventia\ventia-monorepo\apps\backend
✓ Modules imported successfully


## Test 1: OAuth2 Authentication

Probamos la autenticación con eFact-OSE usando OAuth2.

In [2]:
try:
    print("Attempting to authenticate with eFact-OSE...\n")
    token = efact_client._get_token()
    
    if token:
        print("✓ Authentication successful!")
        print(f"  Token (first 30 chars): {token[:30]}...")
        print(f"  Token length: {len(token)} characters")
    else:
        print("✗ Authentication failed: No token returned")
        
except EFactAuthError as e:
    print(f"✗ Authentication failed: {e}")
    print("\nℹ️  Please check your .env file:")
    print("   - EFACT_RUC_VENTIA should be your RUC (11 digits)")
    print("   - EFACT_PASSWORD_REST should be your REST API password")
except Exception as e:
    print(f"✗ Unexpected error: {e}")

Attempting to authenticate with eFact-OSE...

✓ Authentication successful!
  Token (first 30 chars): eyJhbGciOiJIUzI1NiIsInR5cCI6Ik...
  Token length: 301 characters


## Test 2: Number to Words Conversion

Probamos la conversión de números a letras según las reglas de SUNAT.

In [3]:
test_cases = [
    (0.00, "PEN"),
    (1.00, "PEN"),
    (25.75, "PEN"),
    (100.00, "PEN"),
    (150.50, "PEN"),
    (1000.00, "USD"),
    (1234.56, "PEN"),
    (999999.99, "PEN"),
]

print("Testing number to words conversion:\n")
for numero, moneda in test_cases:
    result = numero_a_letras(numero, moneda)
    print(f"{numero:>10} {moneda} → {result}")

Testing number to words conversion:

       0.0 PEN → CERO CON 00/100 SOLES
       1.0 PEN → UNO CON 00/100 SOL
     25.75 PEN → VEINTICINCO CON 75/100 SOLES
     100.0 PEN → CIEN CON 00/100 SOLES
     150.5 PEN → CIENTO CINCUENTA CON 50/100 SOLES
    1000.0 USD → UN MIL CON 00/100 DÓLARES AMERICANOS
   1234.56 PEN → UN MIL DOSCIENTOS TREINTA Y CUATRO CON 56/100 SOLES
 999999.99 PEN → NOVECIENTOS NOVENTA Y NUEVE MIL NOVECIENTOS NOVENTA Y NUEVE CON 99/100 SOLES


## Test 3: Document Validations

Probamos las validaciones de RUC y DNI.

In [4]:
print("RUC Validation Tests:\n")

ruc_tests = [
    ("20551093035", "Valid RUC (11 digits)"),
    ("12345678901", "Valid RUC (11 digits)"),
    ("123456789", "Invalid - only 9 digits"),
    ("205510930355", "Invalid - 12 digits"),
    ("2055109303A", "Invalid - contains letter"),
]

for ruc, description in ruc_tests:
    result = validar_ruc(ruc)
    status = "✓" if result else "✗"
    print(f"{status} {description}: '{ruc}' → {result}")

print("\nDNI Validation Tests:\n")

dni_tests = [
    ("12345678", "Valid DNI (8 digits)"),
    ("87654321", "Valid DNI (8 digits)"),
    ("1234567", "Invalid - only 7 digits"),
    ("123456789", "Invalid - 9 digits"),
    ("1234567A", "Invalid - contains letter"),
]

for dni, description in dni_tests:
    result = validar_dni(dni)
    status = "✓" if result else "✗"
    print(f"{status} {description}: '{dni}' → {result}")

RUC Validation Tests:

✓ Valid RUC (11 digits): '20551093035' → True
✓ Valid RUC (11 digits): '12345678901' → True
✗ Invalid - only 9 digits: '123456789' → False
✗ Invalid - 12 digits: '205510930355' → False
✗ Invalid - contains letter: '2055109303A' → False

DNI Validation Tests:

✓ Valid DNI (8 digits): '12345678' → True
✓ Valid DNI (8 digits): '87654321' → True
✗ Invalid - only 7 digits: '1234567' → False
✗ Invalid - 9 digits: '123456789' → False
✗ Invalid - contains letter: '1234567A' → False


## Test 4: JSON-UBL 2.1 Generation - Boleta (Receipt)

Generamos un JSON-UBL 2.1 completo para una Boleta de Venta (tipo 03).

**IMPORTANTE:** El JSON-UBL 2.1 usa sufijos específicos:
- **Content** para valores (TextContent, CodeContent, AmountContent, etc.)
- **Identifier** para códigos (AmountCurrencyIdentifier, QuantityUnitCode, etc.)
- Incluye todos los metadatos de catálogos SUNAT (URNs, CodeListNameText, etc.)

In [ ]:
# Items de la boleta
items = [
    {
        "sku": "PROD001",
        "description": "Laptop HP Pavilion 15",
        "quantity": 1,
        "unit_price": 2500.00,
        "unit": "NIU",
    },
    {
        "sku": "PROD002",
        "description": "Mouse Logitech M185",
        "quantity": 2,
        "unit_price": 45.00,
        "unit": "NIU",
    },
]

# Cálculo de totales
subtotal = 2590.00  # 2500 + 90
igv = 466.20  # 18%
total = 3056.20

# Generar JSON-UBL 2.1 con formato completo SUNAT
json_ubl = generate_json_ubl(
    invoice_type="03",
    serie="B001",
    correlativo=1,
    fecha_emision=datetime(2026, 1, 8, 14, 30, 0),
    emisor_ruc="20612141453",
    emisor_razon_social="DUVERA COSMETICS S.A.C.",
    emisor_nombre_comercial="LA DORE",
    emisor_ubigeo="150101",
    emisor_departamento="LIMA",
    emisor_provincia="LIMA",
    emisor_distrito="LIMA",
    emisor_direccion="AV. EJEMPLO 123 - LIMA",
    cliente_tipo_doc="1",
    cliente_numero_doc="00000000",
    cliente_razon_social="NINGUNO",
    currency="PEN",
    items=items,
    subtotal=subtotal,
    igv=igv,
    total=total,
)

print("✓ JSON-UBL 2.1 generated successfully\n")
print("Structure Overview:")
print(f"  - Namespaces: {list(json_ubl.keys())[:4]}")
print(f"  - Root Element: Invoice")
print(f"  - Document: {json_ubl['Invoice'][0]['ID'][0]['IdentifierContent']}")
print(f"  - Date: {json_ubl['Invoice'][0]['IssueDate'][0]['DateContent']} {json_ubl['Invoice'][0]['IssueTime'][0]['DateTimeContent']}")
print(f"  - Issuer: {json_ubl['Invoice'][0]['AccountingSupplierParty'][0]['Party'][0]['PartyLegalEntity'][0]['RegistrationName'][0]['TextContent']}")
print(f"  - Customer: {json_ubl['Invoice'][0]['AccountingCustomerParty'][0]['Party'][0]['PartyLegalEntity'][0]['RegistrationName'][0]['TextContent']}")
print(f"  - Total: {json_ubl['Invoice'][0]['LegalMonetaryTotal'][0]['PayableAmount'][0]['AmountContent']} {json_ubl['Invoice'][0]['LegalMonetaryTotal'][0]['PayableAmount'][0]['AmountCurrencyIdentifier']}")
print(f"  - Items: {json_ubl['Invoice'][0]['LineCountNumeric'][0]['NumericContent']}")
print(f"  - Amount in words: {json_ubl['Invoice'][0]['Note'][0]['TextContent']}")
print(f"  - Signature ID: {json_ubl['Invoice'][0]['Signature'][0]['ID'][0]['IdentifierContent']}")

✓ JSON-UBL 2.1 generated successfully

Structure Overview:
  - Namespaces: ['_D', '_S', '_B', '_E']
  - Root Element: Invoice
  - Document: B001-00000004
  - Date: 2026-01-08 14:30:00
  - Issuer: DUVERA COSMETICS S.A.C.
  - Customer: NINGUNO
  - Total: 3056.20 PEN
  - Items: 2
  - Amount in words: TRES MIL CINCUENTA Y SEIS CON 20/100 SOLES
  - Signature ID: IDSignature


### Ver el JSON-UBL 2.1 completo

Nota: Este es el formato XML-like que espera eFact-OSE, con namespaces y estructura UBL 2.1.

In [6]:
# Mostrar estructura del JSON UBL 2.1
print("JSON-UBL 2.1 Structure:\n")
print(f"Namespaces:")
print(f"  _D: {json_ubl['_D']}")
print(f"  _S: {json_ubl['_S']}")
print(f"  _B: {json_ubl['_B']}")
print(f"  _E: {json_ubl['_E']}")
print(f"\nInvoice Root Keys: {list(json_ubl['Invoice'][0].keys())[:10]}...")

# Mostrar InvoiceTypeCode completo (con todos los metadatos SUNAT)
print(f"\nExample: InvoiceTypeCode with SUNAT metadata:")
print(json.dumps(json_ubl['Invoice'][0]['InvoiceTypeCode'][0], indent=2, ensure_ascii=False))

# Mostrar un item completo como ejemplo
print(f"\nExample Invoice Line (Item 1) - First few fields:")
item_keys = list(json_ubl['Invoice'][0]['InvoiceLine'][0].keys())
print(f"Item keys: {item_keys}")
print(f"\nInvoicedQuantity structure:")
print(json.dumps(json_ubl['Invoice'][0]['InvoiceLine'][0]['InvoicedQuantity'][0], indent=2, ensure_ascii=False))

JSON-UBL 2.1 Structure:

Namespaces:
  _D: urn:oasis:names:specification:ubl:schema:xsd:Invoice-2
  _S: urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2
  _B: urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2
  _E: urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2

Invoice Root Keys: ['UBLVersionID', 'CustomizationID', 'ID', 'IssueDate', 'IssueTime', 'InvoiceTypeCode', 'Note', 'DocumentCurrencyCode', 'LineCountNumeric', 'Signature']...

Example: InvoiceTypeCode with SUNAT metadata:
{
  "CodeContent": "03",
  "CodeListNameText": "Tipo de Documento",
  "CodeListSchemeUniformResourceIdentifier": "urn:pe:gob:sunat:cpe:see:gem:catalogos:catalogo51",
  "CodeListIdentifier": "0101",
  "CodeNameText": "Tipo de Operacion",
  "CodeListUniformResourceIdentifier": "urn:pe:gob:sunat:cpe:see:gem:catalogos:catalogo01",
  "CodeListAgencyNameText": "PE:SUNAT"
}

Example Invoice Line (Item 1) - First few fields:
Item keys: ['ID', 'Invoice

## Test 5: JSON-UBL 2.1 Generation - Factura (Invoice)

Generamos un JSON-UBL 2.1 para una Factura (tipo 01).

In [3]:
items_factura = [
    {
        "sku": "SRV001",
        "description": "Servicio de Consultoría TI - 40 horas",
        "quantity": 40,
        "unit_price": 150.00,
        "unit": "HUR",  # Hora
    },
]

subtotal = 6000.00
igv = 1080.00
total = 7080.00

json_ubl_factura = generate_json_ubl(
    invoice_type="01",
    serie="F001",
    correlativo=456,
    fecha_emision=datetime.now(),
    emisor_ruc="20614382741",
    emisor_razon_social="VENTIA SAC",
    emisor_nombre_comercial="VENTIA",
    emisor_direccion="AV. JAVIER PRADO 123 - SAN ISIDRO",
    cliente_tipo_doc="6",  # RUC
    cliente_numero_doc="20100035121",
    cliente_razon_social="EMPRESA CLIENTE SAC",
    currency="PEN",
    items=items_factura,
    subtotal=subtotal,
    igv=igv,
    total=total,
)

print("✓ Factura JSON-UBL 2.1 generated successfully\n")
print(f"Document: {json_ubl_factura['Invoice'][0]['ID'][0]['IdentifierContent']}")
print(f"Customer RUC: {json_ubl_factura['Invoice'][0]['AccountingCustomerParty'][0]['Party'][0]['PartyIdentification'][0]['ID'][0]['IdentifierContent']}")
print(f"Total: {json_ubl_factura['Invoice'][0]['LegalMonetaryTotal'][0]['PayableAmount'][0]['AmountContent']} PEN")
print(f"Amount in words: {json_ubl_factura['Invoice'][0]['Note'][0]['TextContent']}")

✓ Factura JSON-UBL 2.1 generated successfully

Document: F001-00000456
Customer RUC: 20100035121
Total: 7080.00 PEN
Amount in words: SIETE MIL OCHENTA CON 00/100 SOLES


## Test 6: JSON-UBL 2.1 Generation - Nota de Crédito (Credit Note)

Generamos un JSON-UBL 2.1 para una Nota de Crédito (tipo 07) que referencia una boleta anterior.

In [ ]:
items_nc = [
    {
        "sku": "PROD001",
        "description": "Devolución - Laptop HP Pavilion 15 (Producto defectuoso)",
        "quantity": 1,
        "unit_price": 2500.00,
        "unit": "NIU",
    },
]

subtotal = 2500.00
igv = 450.00
total = 2950.00

json_ubl_nc = generate_json_ubl(
    invoice_type="07",
    serie="BC01",
    correlativo=5,
    fecha_emision=datetime.now(),
    emisor_ruc="20551093035",
    emisor_razon_social="VENTIA SAC",
    emisor_nombre_comercial="VENTIA",
    cliente_tipo_doc="1",
    cliente_numero_doc="12345678",
    cliente_razon_social="JUAN PEREZ LOPEZ",
    currency="PEN",
    items=items_nc,
    subtotal=subtotal,
    igv=igv,
    total=total,
    # Referencia al documento original
    reference_type="03",
    reference_serie="B001",
    reference_correlativo=123,
    reference_reason="Devolución de producto defectuoso",
)

print("✓ Credit Note JSON-UBL 2.1 generated successfully\n")
print(f"Document: {json_ubl_nc['Invoice'][0]['ID'][0]['IdentifierContent']}")
print(f"Total: {json_ubl_nc['Invoice'][0]['LegalMonetaryTotal'][0]['PayableAmount'][0]['AmountContent']} PEN")
print(f"\nReferenced Document:")
print(f"  - Type: {json_ubl_nc['Invoice'][0]['BillingReference'][0]['InvoiceDocumentReference'][0]['DocumentTypeCode'][0]['CodeContent']}")
print(f"  - Number: {json_ubl_nc['Invoice'][0]['BillingReference'][0]['InvoiceDocumentReference'][0]['ID'][0]['IdentifierContent']}")
print(f"  - Reason: {json_ubl_nc['Invoice'][0]['DiscrepancyResponse'][0]['Description'][0]['TextContent']}")

## Test 7: Send Document to eFact (OPTIONAL - REAL API CALL)

⚠️ **ADVERTENCIA**: Este test enviará un documento REAL a eFact-OSE.
Solo ejecuta esto si quieres hacer una prueba real con la API.

Descomenta el código para ejecutarlo.

In [4]:
# ⚠️ DESCOMENTA PARA EJECUTAR
try:
    print("Sending document to eFact-OSE...\n")
    
    #Mostrar información del documento antes de enviar
    invoice_data = json_ubl.get("Invoice", [{}])[0]
    ruc_emisor = invoice_data.get("AccountingSupplierParty", [{}])[0].get("Party", [{}])[0].get("PartyIdentification", [{}])[0].get("ID", [{}])[0].get("IdentifierContent", "")
    tipo_documento = invoice_data.get("InvoiceTypeCode", [{}])[0].get("CodeContent", "")
    numero_documento = invoice_data.get("ID", [{}])[0].get("IdentifierContent", "")
    
    print(f"📄 Document info:")
    print(f"   RUC Emisor: {ruc_emisor}")
    print(f"   Tipo Doc: {tipo_documento}")
    print(f"   Número: {numero_documento}")
    
    if "-" in numero_documento:
        serie, correlativo = numero_documento.split("-", 1)
        filename = f"{ruc_emisor}-{tipo_documento}-{serie}-{correlativo}.json"
        print(f"   Filename: {filename}")
    
    print("\n🚀 Sending to eFact...")
    
    # Usar uno de los JSON-UBL generados arriba
    response = efact_client.send_document(json_ubl)
    
    print("✓ Document sent successfully!")
    print(f"\nResponse:")
    print(json.dumps(response, indent=2, ensure_ascii=False))
    
    if 'ticket' in response:
        ticket = response['ticket']
        print(f"\n✓ Ticket: {ticket}")
        print(f"  Status: {response.get('status', 'N/A')}")
        
        # Guardar ticket para consultar estado después
        print(f"\n💡 Use this ticket to check status later:")
        print(f"   efact_client.get_document_status('{ticket}')")
        
except EFactError as e:
    print(f"✗ Error sending document: {e}")
except Exception as e:
    print(f"✗ Unexpected error: {e}")

print("⚠️ Test commented out for safety. Uncomment to run.")
print("\nℹ️  Note: The document will be sent as multipart/form-data")
print("   Filename format: {RUC}-{TIPO_DOC}-{SERIE}-{CORRELATIVO}.json")
print("   Example: 20551093035-03-B001-00000123.json")
print("\n⚠️  Error 0110 'No se pudo obtener la información del tipo de usuario' puede significar:")
print("   1. El RUC del emisor no está registrado/habilitado en eFact-OSE")
print("   2. El RUC usado para autenticar (EFACT_RUC_VENTIA) debe ser el mismo del emisor")
print("   3. Verificar que el formato del JSON sea exactamente el esperado por eFact")

Sending document to eFact-OSE...

📄 Document info:
   RUC Emisor: 20612141453
   Tipo Doc: 03
   Número: B001-00000004
   Filename: 20612141453-03-B001-00000004.json

🚀 Sending to eFact...
✓ Document sent successfully!

Response:
{
  "code": "0",
  "description": "9e92bf85-f31c-4f57-ac6c-b550a52964fb"
}
⚠️ Test commented out for safety. Uncomment to run.

ℹ️  Note: The document will be sent as multipart/form-data
   Filename format: {RUC}-{TIPO_DOC}-{SERIE}-{CORRELATIVO}.json
   Example: 20551093035-03-B001-00000123.json

⚠️  Error 0110 'No se pudo obtener la información del tipo de usuario' puede significar:
   1. El RUC del emisor no está registrado/habilitado en eFact-OSE
   2. El RUC usado para autenticar (EFACT_RUC_VENTIA) debe ser el mismo del emisor
   3. Verificar que el formato del JSON sea exactamente el esperado por eFact


## Test 8: Check Document Status (OPTIONAL - REAL API CALL)

Si tienes un ticket de un documento enviado, puedes consultar su estado.

In [19]:
# # ⚠️ DESCOMENTA Y REEMPLAZA EL TICKET
ticket = "70627243-6ebd-4ebf-90e7-365b08bd2a73"  # Reemplaza con un ticket real

try:
    print(f"Checking status for ticket: {ticket}\n")
    
    status_response = efact_client.get_document_status(ticket)
    
    print("Response:")
    print(json.dumps(status_response, indent=2, ensure_ascii=False))
    
    status = status_response.get('status')
    
    if status == 'processing':
        print("\n⏳ Document is still being processed by SUNAT")
    elif status == 'success':
        print("\n✓ Document successfully validated by SUNAT!")
        if 'cdr' in status_response:
            print("\nCDR (Constancia de Recepción):")
            print(json.dumps(status_response['cdr'], indent=2, ensure_ascii=False))
    elif status == 'error':
        print("\n✗ Document rejected by SUNAT")
        if 'error' in status_response:
            print("\nError details:")
            print(json.dumps(status_response['error'], indent=2, ensure_ascii=False))
    
except EFactError as e:
    print(f"✗ Error checking status: {e}")
except Exception as e:
    print(f"✗ Unexpected error: {e}")

print("⚠️ Test commented out. Uncomment and set a ticket to run.")

Checking status for ticket: 70627243-6ebd-4ebf-90e7-365b08bd2a73

✗ Error checking status: Unexpected status code 404: {"timestamp":1767981186185,"status":404,"error":"Not Found","message":"No message available","path":"/api-efact-ose/v1/document/70627243-6ebd-4ebf-90e7-365b08bd2a73"}
⚠️ Test commented out. Uncomment and set a ticket to run.


## Resumen de Tests

Este notebook ha probado exitosamente:

**✓ OAuth2 Authentication** - Conexión con eFact-OSE  
**✓ Number to Words** - Conversión de números a letras según normativa SUNAT  
**✓ Document Validation** - Validación de RUC y DNI  
**✓ JSON-UBL 2.1 Generation** - Generación de documentos electrónicos en formato UBL 2.1 COMPLETO SUNAT
  - Boleta (03)
  - Factura (01)
  - Nota de Crédito (07) con referencias

### Formato JSON-UBL 2.1 Correcto

El formato JSON-UBL 2.1 implementado ahora cumple COMPLETAMENTE con los estándares SUNAT:

**Sufijos específicos:**
- `IdentifierContent` - Para IDs y códigos de identificación
- `TextContent` - Para textos libres y nombres
- `CodeContent` - Para códigos de catálogo
- `AmountContent` - Para montos monetarios
- `AmountCurrencyIdentifier` - Para códigos de moneda
- `DateContent` / `DateTimeContent` - Para fechas y horas
- `NumericContent` - Para valores numéricos
- `QuantityContent` / `QuantityUnitCode` - Para cantidades y unidades

**Metadatos SUNAT completos:**
- Todos los códigos incluyen referencias a catálogos SUNAT (URNs)
- `CodeListNameText` - Nombre del catálogo
- `CodeListAgencyNameText` - Agencia emisora (PE:SUNAT, PE:INEI, etc.)
- `CodeListUniformResourceIdentifier` - URN del catálogo

**Campos adicionales implementados:**
- `Signature` - Firma digital con referencia
- `LineCountNumeric` - Número de líneas del comprobante
- `PartyName` - Nombre comercial del emisor
- `RegistrationAddress` - Dirección completa con ubigeo
- `Percent` - Porcentaje de IGV en cada ítem
- `AddressTypeCode`, `CityName`, `CountrySubentity`, `District` - Datos de dirección completos

**Formato de número de documento:**
- Ahora usa padding de 8 dígitos: `B001-00000123` en lugar de `B001-123`

### Integración con eFact-OSE

**Envío de documentos:**
- Se envía como `multipart/form-data` con key `file`
- Nombre del archivo: `{RUC}-{TIPO_DOC}-{SERIE}-{CORRELATIVO}.json`
- Ejemplo: `20551093035-03-B001-00000123.json`

**Descarga de archivos:**
- `download_pdf(ticket)` - Retorna bytes del PDF
- `download_xml(ticket)` - Retorna bytes del XML firmado

### Siguiente paso

El cliente eFact (`efact_client.py`) ahora:
- ✅ Genera JSON-UBL 2.1 100% conforme a SUNAT
- ✅ Envía documentos como multipart/form-data con nombre correcto
- ✅ Descarga PDF y XML como bytes

Para continuar con la implementación según el plan (PLAN_EFACT_HISTORIAS_USUARIO.md):

1. **ÉPICA 5**: Crear los repositories (US-020 a US-022)
2. **ÉPICA 6**: Crear los servicios de negocio (US-023 a US-025)  
3. **ÉPICA 7**: Crear los endpoints API (US-026 a US-034)

**LISTO PARA PRUEBAS REALES CON EFACT-OSE** ✓

In [ ]:
# Verificar datos clave del JSON-UBL generado
print("🔍 Verificación del JSON-UBL generado:\n")

invoice_data = json_ubl.get("Invoice", [{}])[0]

# RUC Emisor
ruc_emisor = invoice_data.get("AccountingSupplierParty", [{}])[0].get("Party", [{}])[0].get("PartyIdentification", [{}])[0].get("ID", [{}])[0].get("IdentifierContent", "")
print(f"✓ RUC Emisor: {ruc_emisor}")

# Tipo de documento
tipo_documento = invoice_data.get("InvoiceTypeCode", [{}])[0].get("CodeContent", "")
print(f"✓ Tipo Documento: {tipo_documento} ({'Boleta' if tipo_documento == '03' else 'Factura' if tipo_documento == '01' else 'NC' if tipo_documento == '07' else 'ND'})")

# Número de documento
numero_documento = invoice_data.get("ID", [{}])[0].get("IdentifierContent", "")
print(f"✓ Número Documento: {numero_documento}")

# Filename que se generará
if "-" in numero_documento:
    serie, correlativo = numero_documento.split("-", 1)
    filename = f"{ruc_emisor}-{tipo_documento}-{serie}-{correlativo}.json"
    print(f"✓ Filename: {filename}")

# Cliente
cliente_doc = invoice_data.get("AccountingCustomerParty", [{}])[0].get("Party", [{}])[0].get("PartyIdentification", [{}])[0].get("ID", [{}])[0].get("IdentifierContent", "")
cliente_nombre = invoice_data.get("AccountingCustomerParty", [{}])[0].get("Party", [{}])[0].get("PartyLegalEntity", [{}])[0].get("RegistrationName", [{}])[0].get("TextContent", "")
print(f"✓ Cliente: {cliente_nombre} ({cliente_doc})")

# Total
total_amount = invoice_data.get("LegalMonetaryTotal", [{}])[0].get("PayableAmount", [{}])[0].get("AmountContent", "")
currency = invoice_data.get("LegalMonetaryTotal", [{}])[0].get("PayableAmount", [{}])[0].get("AmountCurrencyIdentifier", "")
print(f"✓ Total: {currency} {total_amount}")

# Número de items
line_count = invoice_data.get("LineCountNumeric", [{}])[0].get("NumericContent", 0)
print(f"✓ Items: {line_count}")

print("\n⚠️  IMPORTANTE:")
print(f"   El RUC usado para autenticar (EFACT_RUC_VENTIA) es: {efact_client.ruc_ventia}")
print(f"   El RUC del emisor en el documento es: {ruc_emisor}")
if efact_client.ruc_ventia != ruc_emisor:
    print(f"   ⚠️  WARNING: Los RUCs NO coinciden! Esto puede causar el error 0110")
else:
    print(f"   ✓ Los RUCs coinciden")

🔍 Verificación del JSON-UBL generado:

✓ RUC Emisor: 20551093035
✓ Tipo Documento: 03 (Boleta)
✓ Número Documento: B001-00000002
✓ Filename: 20551093035-03-B001-00000002.json
✓ Cliente: JUAN PEREZ LOPEZ (72299789)
✓ Total: PEN 3056.20
✓ Items: 2

⚠️  IMPORTANTE:
   El RUC usado para autenticar (EFACT_RUC_VENTIA) es: 20614382741
   El RUC del emisor en el documento es: 20551093035
   ⚠️  WARNING: Los RUCs NO coinciden! Esto puede causar el error 0110


## Test 9: Verificar JSON generado antes de enviar

Verifica que el JSON generado tenga el formato correcto antes de enviarlo a eFact.

In [ ]:
# Crear un ejemplo simple para ver la estructura completa
simple_items = [
    {
        "sku": "PROD001",
        "description": "Producto de Ejemplo",
        "quantity": 1,
        "unit_price": 100.00,
        "unit": "NIU",
    },
]

simple_json_ubl = generate_json_ubl(
    invoice_type="03",
    serie="B999",
    correlativo=1,
    fecha_emision=datetime(2026, 1, 9, 10, 0, 0),
    emisor_ruc="20551093035",
    emisor_razon_social="VENTIA SAC",
    cliente_tipo_doc="1",
    cliente_numero_doc="12345678",
    cliente_razon_social="CLIENTE EJEMPLO",
    currency="PEN",
    items=simple_items,
    subtotal=100.00,
    igv=18.00,
    total=118.00,
)

print("✓ Simple example generated")
print(f"Document: {simple_json_ubl['Invoice'][0]['ID'][0]['IdentifierContent']}")
print(f"Total: {simple_json_ubl['Invoice'][0]['LegalMonetaryTotal'][0]['PayableAmount'][0]['AmountContent']} PEN")

✓ Simple example generated
Document: B999-00000001
Total: 118.00 PEN


### Ver JSON-UBL 2.1 completo (FULL)

Descomenta cualquiera de las opciones para ver el JSON completo de un documento.

In [8]:
# Crear un ejemplo simple para ver la estructura completa
simple_items = [
    {
        "sku": "PROD001",
        "description": "Producto de Ejemplo",
        "quantity": 1,
        "unit_price": 100.00,
        "unit": "NIU",
    },
]

simple_json_ubl = generate_json_ubl(
    invoice_type="03",
    serie="B999",
    correlativo=1,
    fecha_emision=datetime(2026, 1, 9, 10, 0, 0),
    emisor_ruc="20551093035",
    emisor_razon_social="VENTIA SAC",
    cliente_tipo_doc="1",
    cliente_numero_doc="12345678",
    cliente_razon_social="CLIENTE EJEMPLO",
    currency="PEN",
    items=simple_items,
    subtotal=100.00,
    igv=18.00,
    total=118.00,
)

print("Estructura completa JSON-UBL 2.1:\n")
print(json.dumps(simple_json_ubl, indent=2, ensure_ascii=False))

Estructura completa JSON-UBL 2.1:

{
  "_D": "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2",
  "_S": "urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2",
  "_B": "urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2",
  "_E": "urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
  "Invoice": [
    {
      "UBLVersionID": [
        {
          "IdentifierContent": "2.1"
        }
      ],
      "CustomizationID": [
        {
          "IdentifierContent": "2.0"
        }
      ],
      "ID": [
        {
          "IdentifierContent": "B999-00000001"
        }
      ],
      "IssueDate": [
        {
          "DateContent": "2026-01-09"
        }
      ],
      "IssueTime": [
        {
          "DateTimeContent": "10:00:00"
        }
      ],
      "InvoiceTypeCode": [
        {
          "CodeContent": "03",
          "CodeListNameText": "Tipo de Documento",
          "CodeListSchemeUniformResourceIdentifier": "urn: